# Resume-Based Job Query Generator

This notebook analyzes resumes (PDF/DOC) and generates relevant LinkedIn job search queries based on skills and experience.


## Import Required Libraries


In [24]:
import os
import sys
from pathlib import Path
from typing import List, Optional

from langchain_ollama import OllamaLLM
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from pydantic import BaseModel, Field
from dotenv import find_dotenv, load_dotenv


# Add parent directory to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("."))))

In [25]:
# Load environment variables
load_dotenv(find_dotenv())

True

## Setup Ollama LLM


In [ ]:
# Initialize OpenAI LLM with GPT-3.5 Turbo
from langchain_openai import ChatOpenAI
import os

# # Load OpenAI API key from environment variable
# openai_api_key = os.getenv("OPENAI_API_KEY")
# if not openai_api_key:
#     raise ValueError("OPENAI_API_KEY environment variable is not set")

# llm = ChatOpenAI(model="gpt-5", temperature=0.1)
llm = ChatOpenAI(
    model="gpt-5",
    # "gpt-4o-mini",
    # "gpt-5",
    # "gpt-3.5-turbo"
    temperature=0.3,
)
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=1000)

# Commented out Ollama configuration
# llm = OllamaLLM(
#     model="llama3.2",
#     base_url="http://localhost:11434",
#     temperature=0.1,
#     num_predict=1000,
# )

## Define Output Schema


In [34]:
class JobSearchQueries(BaseModel):
    """Structured output for job search queries."""

    primary_titles: List[str] = Field(
        description="Realistic next-step job titles with best chance of getting hired (5 titles)"
    )
    secondary_titles: List[str] = Field(
        description="Opportunistic and futuristic job titles leveraging domain/technical knowledge (8 titles)"
    )
    skill_based_queries: List[str] = Field(
        description="Skill-focused search queries (2-3 queries)"
    )
    industry_focus: str = Field(
        description="Primary industry or sector based on experience"
    )
    seniority_level: str = Field(
        description="Experience level: Entry, Mid, Senior, or Executive"
    )

## Create System and User Prompts


In [35]:
SYSTEM_PROMPT = """
You are an expert career advisor and recruitment specialist specializing in future career growth and opportunities. Your task is to analyze a resume and generate forward-looking LinkedIn job search queries that focus on career advancement and future potential.

Analyze the resume content and focus on:
1. Technical skills and competencies - what roles leverage these best?
2. Work experience and career progression - identify patterns and strengths
3. Industry background - current expertise and emerging sectors
4. Educational qualifications - how can these be leveraged?
5. Certifications and achievements - what doors do these open?

IMPORTANT DISTINCTION:

PRIMARY JOB TITLES (5 titles): Focus on realistic next-step opportunities where the candidate has the BEST CHANCE of getting hired based on their current strengths and experience. Give preference to roles that build naturally from their most recent positions and demonstrated expertise. These should be achievable career moves.

SECONDARY JOB TITLES (8 titles): Focus on more FUTURISTIC and OPPORTUNISTIC roles that leverage the candidate's domain knowledge and technical skills in new ways. Include:
- Emerging positions and innovative job titles
- Cross-industry opportunities where their skills transfer
- Advanced roles that represent significant career leaps
- Opportunistic pivots based on technical knowledge (consulting, product, strategy)
- Domain expertise applications in new contexts
- Leadership and advisory roles based on technical background
- Entrepreneurial or specialized consulting opportunities
- Think 2-5 years into the future and consider evolving job markets

IMPORTANT FORMATTING RULES:
- Provide ONLY the job title without any explanations, justifications, or descriptions
- Do NOT include seniority levels (Senior, Junior, Lead, etc.) in the title - keep titles generalized
- Keep titles clean and searchable on LinkedIn

Format your response exactly as follows:

PRIMARY JOB TITLES:
- [job title 1]
- [job title 2]
- [job title 3]
- [job title 4]
- [job title 5]

SECONDARY JOB TITLES:
- [futuristic role 1]
- [cross-industry opportunity 2]
- [advanced leadership position 3]
- [innovative field role 4]
- [consulting/advisory role 5]
- [domain expertise pivot 6]
- [entrepreneurial opportunity 7]
- [specialized expertise role 8]

SKILL-BASED QUERIES:
- [skill combination 1]
- [skill combination 2]

INDUSTRY FOCUS: [primary industry based on experience + emerging opportunities]

SENIORITY LEVEL: [Target future level: Mid/Senior/Executive/Leadership]
"""

USER_PROMPT = """
Based on the following resume content, generate job search queries for LinkedIn with clear distinction between realistic next steps and opportunistic future roles:

RESUME CONTENT:
{resume_content}

Analyze this resume and provide:
1. PRIMARY TITLES: Realistic roles where they have the best chance of getting hired (bias toward recent experience and proven skills)
2. SECONDARY TITLES: Opportunistic and futuristic roles that leverage their domain knowledge and technical skills in creative ways - include consulting, advisory, cross-industry pivots, and entrepreneurial opportunities

Remember: Provide ONLY clean job titles without explanations or seniority levels. Keep titles generalized and LinkedIn-searchable.

Please provide the job search information in the exact format specified above.
"""

# Create prompt template
prompt_template = PromptTemplate(
    template=SYSTEM_PROMPT + "\n" + USER_PROMPT, input_variables=["resume_content"]
)

## Build LangChain with Structured Output


In [ ]:
import json
import re
from langchain.schema import BaseOutputParser


class JobQueryOutputParser(BaseOutputParser):
    """Parser to extract structured job queries from LLM responses."""

    def parse(self, text: str) -> JobSearchQueries:
        """Parse job search queries from text."""
        # Initialize lists and variables
        primary_titles = []
        secondary_titles = []
        skill_based_queries = []
        industry_focus = ""
        seniority_level = ""

        lines = [line.strip() for line in text.split("\n") if line.strip()]
        current_section = None

        for line in lines:
            line_lower = line.lower()

            # Section headers
            if "primary" in line_lower and "title" in line_lower:
                current_section = "primary"
                continue
            elif "secondary" in line_lower and "title" in line_lower:
                current_section = "secondary"
                continue
            elif "skill" in line_lower and (
                "query" in line_lower or "queries" in line_lower
            ):
                current_section = "skills"
                continue
            elif "industry" in line_lower and "focus" in line_lower:
                current_section = "industry"
                # Extract industry from same line if present
                if ":" in line:
                    industry_focus = line.split(":", 1)[1].strip()
                continue
            elif "seniority" in line_lower and "level" in line_lower:
                current_section = "seniority"
                # Extract seniority from same line if present
                if ":" in line:
                    seniority_level = line.split(":", 1)[1].strip()
                continue

            # Extract list items
            if line.startswith(
                (
                    "- ",
                    "• ",
                    "* ",
                    "1. ",
                    "2. ",
                    "3. ",
                    "4. ",
                    "5. ",
                    "6. ",
                    "7. ",
                    "8. ",
                )
            ):
                # Remove list markers
                item = re.sub(r"^[-•*]?\s*\d*\.?\s*", "", line).strip()
                if item:
                    if current_section == "primary":
                        primary_titles.append(item)
                    elif current_section == "secondary":
                        secondary_titles.append(item)
                    elif current_section == "skills":
                        skill_based_queries.append(item)

            # Extract key-value pairs
            elif ":" in line and current_section in ["industry", "seniority"]:
                key, value = line.split(":", 1)
                value = value.strip()
                if "industry" in key.lower() or current_section == "industry":
                    industry_focus = value
                elif (
                    "seniority" in key.lower()
                    or "level" in key.lower()
                    or current_section == "seniority"
                ):
                    seniority_level = value

        # Validate that we have the required data
        if not primary_titles:
            raise ValueError("Failed to extract primary job titles from LLM response")
        if not secondary_titles:
            raise ValueError("Failed to extract secondary job titles from LLM response")
        if not skill_based_queries:
            raise ValueError("Failed to extract skill-based queries from LLM response")
        if not industry_focus:
            raise ValueError("Failed to extract industry focus from LLM response")
        if not seniority_level:
            raise ValueError("Failed to extract seniority level from LLM response")

        return JobSearchQueries(
            primary_titles=primary_titles[:5],
            secondary_titles=secondary_titles[
                :8
            ],  # Updated to allow 8 secondary titles
            skill_based_queries=skill_based_queries[:3],
            industry_focus=industry_focus,
            seniority_level=seniority_level,
        )


# Create parser and chain
parser = JobQueryOutputParser()
job_query_chain = prompt_template | llm | parser

## Create Resume Processing Function


In [37]:
def process_resume(file_path: str) -> JobSearchQueries:
    """Process resume file and generate job search queries."""

    file_path = Path(file_path)

    if not file_path.exists():
        raise FileNotFoundError(f"Resume file not found: {file_path}")

    # Load document based on file extension
    if file_path.suffix.lower() == ".pdf":
        loader = PyPDFLoader(str(file_path))
    elif file_path.suffix.lower() in [".doc", ".docx"]:
        loader = Docx2txtLoader(str(file_path))
    else:
        raise ValueError(f"Unsupported file format: {file_path.suffix}")

    # Load and extract text
    documents = loader.load()
    resume_content = "\n".join([doc.page_content for doc in documents])

    # Generate job queries using the chain
    result = job_query_chain.invoke({"resume_content": resume_content})

    return result

## Test with Sample Resume


In [38]:
# Test with existing resume in the data folder
resume_path = "../genai_job_finder/data/Ali Zarreh_CV_2025_08_30.pdf"

try:
    job_queries = process_resume(resume_path)

    print("🎯 Generated Job Search Queries:")
    print("\n📍 Primary Job Titles:")
    for title in job_queries.primary_titles:
        print(f"  • {title}")

    print("\n🔍 Secondary Job Titles:")
    for title in job_queries.secondary_titles:
        print(f"  • {title}")

    print("\n🛠️ Skill-Based Queries:")
    for query in job_queries.skill_based_queries:
        print(f"  • {query}")

    print(f"\n🏭 Industry Focus: {job_queries.industry_focus}")
    print(f"📈 Seniority Level: {job_queries.seniority_level}")

except Exception as e:
    print(f"Error processing resume: {e}")
    print(
        "Make sure OPENAI_API_KEY environment variable is set and the resume file exists"
    )

🎯 Generated Job Search Queries:

📍 Primary Job Titles:
  • Generative AI Engineer
  • Machine Learning Engineer
  • Applied Scientist
  • Data Science Manager
  • Operations Research Scientist

🔍 Secondary Job Titles:
  • AI Product Manager
  • AI Solutions Architect
  • LLM Platform Engineer
  • Conversational AI Architect
  • AI Governance Specialist
  • AI Strategy Consultant
  • Digital Twin Engineer
  • RAG Engineer

🛠️ Skill-Based Queries:
  • generative ai RAG LangChain LangGraph LLM FastAPI "agentic workflows"
  • Databricks Spark "time series" forecasting XGBoost "price optimization" "assortment optimization" retail

🏭 Industry Focus: Retail and e-commerce analytics + applied AI in supply chain, manufacturing, and enterprise GenAI platforms
📈 Seniority Level: Leadership


## Usage Example with Custom Resume


In [39]:
# Example usage with custom resume path
# Replace with your resume path
custom_resume_path = "/path/to/your/resume.pdf"

# Uncomment to test with your resume:
# job_queries = process_resume(custom_resume_path)
# print(job_queries)